##  Sentinel-3 OLCI Level-2 Tiling

### Service Definition

In [1]:
service = dict([('title', 'Sentinel-3 OLCI level-2 tiling'),
                ('abstract', 'This service takes as input one or more Sentinel-3 OLCI Level 2 (OL_2_LFR____) products and tiles'),
                ('identifier', 'ewf-wfp-03-01-01')])

### Parameter Definition 

### Runtime parameter definition

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [2]:
input_reference = dict([('identifier', 'input_reference'),
                        ('title', 'Sentinel-3 OLCI Level 2 (OL_2_LFR____)'),
                        ('abstract', 'Sentinel-3 OLCI Level 2 (OL_2_LFR____) catalogue reference'),
                        ('value', 'https://catalog.terradue.com/sentinel3/search?uid=S3A_OL_2_LFR____20200612T063015_20200612T063315_20200612T081703_0179_059_191_2700_LN1_O_NR_002'),
                        ('stac:collection', 'input_reference'),
                        ('stac:href', 'catalog.json'),
                        ('max_occurs', '16')])

In [3]:
tiling_level = dict([('identifier', 'tiling_level'),
                ('value', '5'),
                ('title', 'Tiling level'),
                ('abstract', 'Tiling level'),
                ('max_occurs', '1')])

**Data path**

This path defines where the data is staged-in. 

In [4]:
data_path = '/workspace/data/s3'

In [5]:
input_catalog = '/workspace/data/s3/catalog.json'

### Workflow

#### Import the packages

In [6]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'

In [7]:
import os
import sys
os.environ['PREFIX'] = '/opt/anaconda/envs/env_s3/'
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
sys.path.append('.')
import gdal
from helpers import *
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import shape
import shutil
from pystac import Catalog, Collection, Item, MediaType, Asset, CatalogType
from tiling import s3_tiles
import glob
gdal.UseExceptions()

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
cat = Catalog.from_file(input_catalog)

if cat is None:
    raise ValueError()

In [10]:
collection = next(cat.get_children())

In [11]:
item = next(collection.get_items())

### Import Sentinel-3 SLSTR product

In [12]:
operators = ['Read', 
             'Reproject',
             'Write']

In [13]:
read = dict()

s3_path = item.assets['metadata'].get_absolute_href()

read['file'] =  s3_path
read['formatName'] = 'Sen3'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

write = dict()
write['file'] = 's3_olci'

In [14]:
snap_graph(os.environ['GPT_BIN'],
           operators,
           Read=read, 
           Reproject=reproject,
           Write=write)

In [18]:
def s3_to_tile(input_tif, item, tile):
    
    translate_options = gdal.TranslateOptions(gdal.ParseCommandLine("-co TILED=YES -co COPY_SRC_OVERVIEWS=YES -co COMPRESS=LZW"))
        
    x_min, y_min, x_max, y_max = tile.tile.bounds

    output_tile_name = '{}_L{}_C{}_R{}'.format(item.id,
                                                    tile.level,
                                                    tile.col,
                                                    tile.row)


    gdal.Translate('tmp_{}.tif'.format(output_tile_name),
                   input_tif,
                   projWin=[x_min, y_max, x_max, y_min],
                   projWinSRS='EPSG:4326')

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name),
                   gdal.OF_READONLY)

    gdal.SetConfigOption('COMPRESS_OVERVIEW', 'DEFLATE')
    ds.BuildOverviews('NEAREST', [2,4,8,16,32])
    ds = None

    ds = gdal.Open('tmp_{}.tif'.format(output_tile_name))
    ds = gdal.Translate('{}.tif'.format(output_tile_name), ds, options=translate_options)
    ds = None

    band_names = ['NDVI', 'OGVI', 'OCTI', 'Land mask', 'Cloud mask', 'OGVI fail mask']

#     ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)

#     # update extended area to -10000 instead of 0 as gdal.trasnlate does
#     ndvi_data = ds.GetRasterBand(1).ReadAsArray()
#     land_data = ds.GetRasterBand(2).ReadAsArray()
#     cloud_data = ds.GetRasterBand(3).ReadAsArray()
    
#     # If land_mask=0 we have water if cloud_mask=1 we have cloud
    
#     masked_ndvi = lambda x,y,z: 255 if x==0 or y==1  else z
#     vfunc_masked_ndvi = np.vectorize(masked_ndvi, otypes=[np.float])
    
#     updated_ndvi_data = vfunc_masked_ndvi(land_data, cloud_data,ndvi_data)
    
    
#     ds.GetRasterBand(1).WriteArray(updated_ndvi_data)
    
#     ds.FlushCache()
    
#     ds = None
    
    ds = gdal.Open('{}.tif'.format(output_tile_name), gdal.GA_Update)
    
    for index in range(ds.RasterCount):

        srcband = ds.GetRasterBand(index+1)

        srcband.SetDescription(band_names[index])

        if index == 0:
            
            srcband.SetNoDataValue(255)
    
    ds.FlushCache()

    ds = None
    
    with open(output_tile_name + '.properties', 'w') as file:

        file.write('title=Tile L:{1} C:{2} R:{3} {0}\n'.format(item.id,
                                                              tile.level, 
                                                              tile.col, 
                                                              tile.row))
        
        file.write('date={}/{}\n'.format(item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ'), 
                                         item.datetime.strftime('%Y-%m-%dT%H:%M:%sZ')))
        
        file.write('geometry={0}'.format(tile.tile.wkt))


    for f in ['tmp_{}.tif'.format(output_tile_name), 'tmp_{}.tif.ovr'.format(output_tile_name)]:

        if os.path.exists(f):

            os.remove(f)
    
    return True
    

### Tiling

In [19]:
bands = [os.path.join('s3_olci.data', '{}.img'.format(band)) for band in ['OGVI', 'OTCI', 'RC681', 'RC865', 'LQSF']]
        
s3_data = read_s3(bands)
        

In [20]:
ds = gdal.Open(bands[0])

geo_transform = ds.GetGeoTransform()
projection_ref = ds.GetProjectionRef()
    

In [21]:
ogvi = s3_data[:,:,0]
otci= s3_data[:,:,1]
red = s3_data[:,:,2]
nir = s3_data[:,:,3]
lqsf = s3_data[:,:,4]

In [22]:
mask = get_mask('LAND', lqsf)

In [23]:
cloud_mask =  get_mask('CLOUD', lqsf)

In [24]:
ogvi_fail_mask =  get_mask('OGVI_FAIL', lqsf)

In [25]:
#Calculate NDVI
ndvi_lambda = lambda x,y,z: 255 if(x+y)==0 or z  else  (x-y)/float(x+y)
vfunc_ndvi = np.vectorize(ndvi_lambda, otypes=[np.float])

ndvi=vfunc_ndvi(nir, red, ogvi_fail_mask )


In [26]:
output_name = 'temp.tif'

In [27]:
driver = gdal.GetDriverByName('GTiff')

output = driver.Create(output_name, 
                       ogvi.shape[1], 
                       ogvi.shape[0], 
                       6, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(projection_ref)
output.GetRasterBand(1).WriteArray(ndvi)
output.GetRasterBand(2).WriteArray(ogvi)
output.GetRasterBand(3).WriteArray(otci)
output.GetRasterBand(4).WriteArray(mask)
output.GetRasterBand(5).WriteArray(cloud_mask)
output.GetRasterBand(6).WriteArray(ogvi_fail_mask)

output.FlushCache()

output = None

In [28]:
tiles = s3_tiles(shape(item.geometry), int(tiling_level['value']))


In [29]:
tiles

,col,row,level,s3_tile,tile
0,40,17,5,"POLYGON ((50.62500 9.11607, 50.52500 9.13774, ...","POLYGON ((50.62500 5.62500, 50.62500 11.25000,..."
1,41,17,5,"POLYGON ((56.03660 7.90594, 55.42580 8.04637, ...","POLYGON ((56.25000 5.62500, 56.25000 11.25000,..."
2,42,17,5,"POLYGON ((56.82558 11.25000, 56.65870 10.55470...","POLYGON ((61.87500 5.62500, 61.87500 11.25000,..."
3,40,18,5,"POLYGON ((45.00000 13.69911, 45.37780 15.66690...","POLYGON ((50.62500 11.25000, 50.62500 16.87500..."
4,42,18,5,"POLYGON ((58.20895 16.87500, 57.95040 15.85760...","POLYGON ((61.87500 11.25000, 61.87500 16.87500..."
5,41,18,5,"POLYGON ((56.25000 11.25000, 50.62500 11.25000...","POLYGON ((56.25000 11.25000, 56.25000 16.87500..."
6,40,19,5,"POLYGON ((45.60517 16.87500, 45.87350 18.30070...","POLYGON ((50.62500 16.87500, 50.62500 22.50000..."
7,41,19,5,"POLYGON ((50.62500 20.19255, 50.91960 20.13760...","POLYGON ((56.25000 16.87500, 56.25000 22.50000..."
8,42,19,5,"POLYGON ((56.25000 19.04677, 56.71120 18.94540...","POLYGON ((61.87500 16.87500, 61.87500 22.50000..."
9,39,17,5,"POLYGON ((45.00000 10.27494, 44.97250 10.28040...","POLYGON ((45.00000 5.62500, 45.00000 11.25000,..."


In [30]:
for index, tile in tiles.iterrows():
    
    logging.info('Tile L{} C{} R{}'.format(tile.level,
                                       tile.col,
                                       tile.row))


    s3_to_tile(output_name, item, tile)

In [31]:
logging.info('Clean-up') 
os.remove(output_name)

shutil.rmtree('s3_olci.data')
os.remove('s3_olci.dim')

time.sleep(45)

for f in glob.glob('./*.tif.aux.xml'):

    os.remove(f)


### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.